In [1]:
# Starting PyImageJ, some of these dependencies may be autodiscovered via transitive dependencies, not sure

imagej_core_dep = 'net.imagej:imagej:2.3.0'
imagej_legacy_dep = 'net.imagej:imagej-legacy:0.38.1'
abba_dep = 'ch.epfl.biop:ImageToAtlasRegister:0.2.8'

deps_pack = [imagej_core_dep, imagej_legacy_dep, abba_dep]

In [2]:
# Starts ImageJ, show UI
import imagej
ij = imagej.init(deps_pack, headless=False)
ij.ui().showUI()

In [9]:
# Importing necessary classes from Java for the next cell, the hard one

from scyjava import jimport
import jpype
import jpype.imports
from jpype.types import *
from jpype import JImplements, JOverride

# Starts ABBA

# .. but before : logger, please shut up
DebugTools = jimport('loci.common.DebugTools')
DebugTools.enableLogging('INFO')

# Ok, let's start ABBA and its BDV view (it's also possible to start it without any GUI, 
# or even to build another GUI with a Napari view, why not ?)

ABBABdvStartCommand = jimport('ch.epfl.biop.atlas.aligner.gui.bdv.ABBABdvStartCommand') # Command import
ij.command().run(ABBABdvStartCommand, True) # Starts it with the converted brainglobe atlas in the coronal orientation

<java object 'java.util.concurrent.FutureTask'>

## Download serial sections examples

Download sections 30, 40, 50 from the zenodo repository: https://zenodo.org/record/4715656#.Ybe-8Fko_iE (around 100Mb per section...)

Files are put in the current repository, under the `images` folder. If files have already been downloaded, the download will be skipped.

In [5]:
import os
from bg_atlasapi import utils
from pathlib import Path
cwd = os.getcwd() # gets current path

utils.check_internet_connection()
base_zenodo_url = 'https://zenodo.org/record/4715656/'

basePath = cwd+'/images/'

def downloadIfNecessary(section_name):
    outputPath = Path(basePath+section_name)
    if not outputPath.exists():
        url = 'https://zenodo.org/record/4715656/files/'+section_name+'?download=1'
        utils.retrieve_over_http(url, outputPath)
    
downloadIfNecessary('S30.ome.tiff') #https://zenodo.org/record/4715656/files/S30.ome.tiff?download=1
downloadIfNecessary('S40.ome.tiff') #https://zenodo.org/record/4715656/files/S40.ome.tiff?download=1
downloadIfNecessary('S50.ome.tiff') #https://zenodo.org/record/4715656/files/S50.ome.tiff?download=1


In [6]:
# Let's get the multipositioner object 
MultiSlicePositioner = jimport('ch.epfl.biop.atlas.aligner.MultiSlicePositioner')

# There's only one multipositioner instance in the object service
# https://javadoc.scijava.org/SciJava/org/scijava/object/ObjectService.html
mp = ij.object().getObjects(MultiSlicePositioner).get(0)


In [10]:
# Let's import the files using Bio-Formats.
# The list of all commands is accessible here:
# https://github.com/BIOP/ijp-imagetoatlas/tree/master/src/main/java/ch/epfl/biop/atlas/aligner/command

ImportImageCommand = jimport('ch.epfl.biop.atlas.aligner.command.ImportImageCommand')

# Here we want to import images: check
# https://github.com/BIOP/ijp-imagetoatlas/blob/master/src/main/java/ch/epfl/biop/atlas/aligner/command/ImportImageCommand.java

File = jimport('java.io.File')

file_s30 = File(basePath+'S30.ome.tiff')
file_s40 = File(basePath+'S40.ome.tiff')
file_s50 = File(basePath+'S50.ome.tiff')

FileArray = JArray(File)
files = FileArray(3)

files[0] = file_s30
files[1] = file_s40
files[2] = file_s50

# Any missing input parameter will lead to a popup window asking the missing argument to the user
ij.command().run(ImportImageCommand, True,\
                 "files", files,\
                 "mp", mp,\
                 "split_rgb_channels", False,\
                 "slice_axis_initial", 5.0,\
                 "increment_between_slices", 0.04\
                )


<java object 'java.util.concurrent.FutureTask'>

[java.lang.Enum.toString] could not reopen file - deleting invalid memo file: C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks\images\.S30.ome.tiff.bfmemo
[java.lang.Enum.toString] OMETiffReader initializing C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks\images\S30.ome.tiff
[java.lang.Enum.toString] Reading IFDs
[java.lang.Enum.toString] Populating metadata
[java.lang.Enum.toString] Unknown JPEG 2000 box 0x290000 at 9292
[java.lang.Enum.toString] File is a raw codestream not a JP2.
[java.lang.Enum.toString] Unknown JPEG 2000 box 0x290000 at 89614138
[java.lang.Enum.toString] File is a raw codestream not a JP2.
[java.lang.Enum.toString] Unknown JPEG 2000 box 0x9f9d6c1 at 78079627
[java.lang.Enum.toString] Attempts to set opener settings for file format OME-TIFF; data location = C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks\images\S30.ome.tiff
[java.lang.Enum.toString] No name found for serie 0 ch 0 setting name to ch_0
[java.lang.Enum.toString] No name found for serie 0

[java.lang.Enum.toString] S30.tif-ch0 deselected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S50.tif-ch0 deselected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S30.tif-ch0 selected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S50.tif-ch0 selected[java.lang.Enum.toString] 


In [11]:
mp.selectSlice(mp.getSlices()) # select all slices

In [12]:
mp.getReslicedAtlas().setRotateY(0.05) # Small correction in Y slicing

In [13]:
mp.deselectSlice(mp.getSlices()) # deselect all

[java.lang.Enum.toString] S30.tif-ch0 deselected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S40.tif-ch0 deselected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S50.tif-ch0 deselected[java.lang.Enum.toString] 


In [14]:
mp.selectSlice(mp.getSlices().get(2)) # select the last slice

[java.lang.Enum.toString] S50.tif-ch0 selected[java.lang.Enum.toString] 


In [15]:
# The slices are always sorted from small z to high z. To keep track of who's who, reference them before moving them
slice30 = mp.getSlices().get(0) 
slice40 = mp.getSlices().get(1)
slice50 = mp.getSlices().get(2)

In [17]:
mp.moveSlice(slice50,9.5)
mp.moveSlice(slice40,8.2)
mp.moveSlice(slice30,7.5)

[java.lang.Enum.toString] S50.tif-ch0 z position changed[java.lang.Enum.toString] 
[java.lang.Enum.toString] S30.tif-ch0 z position changed[java.lang.Enum.toString] 
[java.lang.Enum.toString] S40.tif-ch0 z position changed[java.lang.Enum.toString] 


In [18]:
# Simple actions are accessible through mp.whatever, but most actions are executed on selected slices
# Almost all actions are executed asynchronously

# For a registration : let's select all slices
mp.selectSlice(mp.getSlices()) # select all

[java.lang.Enum.toString] S50.tif-ch0 deselected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S30.tif-ch0 selected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S40.tif-ch0 selected[java.lang.Enum.toString] 
[java.lang.Enum.toString] S50.tif-ch0 selected[java.lang.Enum.toString] 


In [19]:
# Let's run an affine registration on the green slice channel and on the reference atlas channel
# elastix needs to be setup, see https://biop.github.io/ijp-imagetoatlas/installation.html
RegistrationElastixAffineCommand = jimport('ch.epfl.biop.atlas.aligner.command.RegistrationElastixAffineCommand')

ij.command().run(RegistrationElastixAffineCommand, True,
                 "mp", mp,\
                 "pixel_size_micrometer", 40,\
                 "show_imageplus_registration_result", False,\
                 "background_offset_value_moving",0,\
                 "atlas_image_channel",0,\
                 "slice_image_channel",1) # second channel, 0-based

<java object 'java.util.concurrent.FutureTask'>

[java.lang.Enum.toString] ch.epfl.biop.wrappers.elastix.RegisterHelper[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.wrappers.elastix.RegisterHelper[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.wrappers.elastix.RegisterHelper[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.wrappers.elastix.RegParamAffine_Fast[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.

In [20]:
# Let's try spline
RegistrationElastixSplineCommand = jimport('ch.epfl.biop.atlas.aligner.command.RegistrationElastixSplineCommand')

ij.command().run(RegistrationElastixSplineCommand, True,
                 "mp", mp,\
                 "nb_control_points_x", 12,\
                 "pixel_size_micrometer", 20,\
                 "show_imageplus_registration_result", False,\
                 "background_offset_value_moving",0,\
                 "atlas_image_channel",0,\
                 "slice_image_channel",1) # second channel, 0-based

<java object 'java.util.concurrent.FutureTask'>

[java.lang.Enum.toString] ch.epfl.biop.wrappers.elastix.RegisterHelper[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.wrappers.elastix.RegisterHelper[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.wrappers.elastix.RegisterHelper[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] Fixed image fixé[java.lang.Enum.toString] 
[java.lang.Enum.toStri

In [21]:
# Let's wait for all registration to finish
mp.waitForTasks()

[java.lang.Enum.toString] Field UseBinaryFormatForTransformationParameters not found in itc.transforms.elastix.ElastixBSplineTransform class.[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.image.ConvertibleImage[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.roi.ConvertibleRois[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.roi.ConvertibleRois[java.lang.Enum.toString] 
[java.lang.Enum.toString] ch.epfl.biop.java.utilities.roi.ConvertibleRois[java.lang.Enum.toString] 
[java.lang.Enum.toString] NR Converting:ch.epfl.biop.java.utilities.roi.types.RealPointList->ch.epfl.biop.java.utilities.roi.types.TransformixInputRoisFile[java.lang.Enum.toString] 
[java.lang.Enum.toString] Found -> ch.epfl.biop.java.utilities.roi.types.TransformixInputRoisFile State[java.lang.Enum.toString] 
[java.lang.Enu

In [35]:
# Get transformation
transformslice30 = slice30.getSlicePixToCCFRealTransform()

DoubleArray = JArray(JDouble)

coordInImage = DoubleArray(3)
coordInCCF = DoubleArray(3)

coordInImage[0] = 5000 # X (pixel)
coordInImage[1] = 5000 # Y
coordInImage[2] = 0 # Z

transformslice30.inverse().apply(coordInImage,coordInCCF)

print('CCF coord (mm):'+str(coordInCCF))


CCF coord (mm):[6.605262550289882, 3.853260714522113, 7.503746379170531]
